In [1]:
import pandas as pd
%config ZMQInteractiveShell.ast_node_interactivity='all'
%pprint
import warnings 
warnings.filterwarnings('ignore')

Pretty printing has been turned OFF


In [2]:
import re
import numpy as np
import pandas as pd

## 读取文本

In [106]:
df_test = pd.read_csv('./data/test.csv',sep="###__###",names=['ID','Query_List'])
df_train = pd.read_csv('./data/train.csv',sep='###__###',names=['ID', 'Age', 'Gender', 'Education', 'Query_List'])

In [107]:
df_test.head()
df_train.head(10)

,ID,Query_List
0,ED89D43B9F602F96D96C25255F7C228C,陈学冬将出的作品\t刘昊然与谭松韵\t211学校的分数线\t谁唱的味道好听\t吻戏是真吻还是...
1,83C3B7B4AAF8074655A8079F561A76D6,e的0.0052次方\tqq怎么快速提现\t绝色倾城飞烟\t马克思主义基本原理概论\t康世恩...
2,CA9F675A024FB2353849350A35CF8B0F,黑暗文\tlpl夏季赛\t大富豪电玩城\t英雄联盟之电竞称王\t手机怎么扫描手机上的二维码\...
3,DE45B5C4E57AAEBCF3FDFA2A774093BF,中秋水库钓鱼\t鱼竿\t用蚯蚓钓鱼怎样调漂\t传统钓\t3号鱼钩\t鲫鱼汤的做法大全\t鱼饵...
4,406A681FB3DF81EC0E561796AE50AE50,号码吉凶\t退休干部死后配偶\t郫县有哪些大学\t胜利油田属于中石化还是中石油\t苏珊米勒狮...


,ID,Age,Gender,Education,Query_List
0,22DD920316420BE2DF8D6EE651BA174B,1,1,4,柔和双沟\t女生\t中财网首页 财经\thttp://pan.baidu.com/s/1pl...
1,43CC3AF5A8D6430A3B572337A889AFE4,2,1,3,"广州厨宝烤箱\t世情薄,人情恶,雨送黄昏花易落,晓风干,泪痕\t厦门酒店用品批发市场\t我只..."
2,E97654BFF5570E2CCD433EA6128EAC19,4,1,0,钻石之泪耳机\t盘锦到沈阳\t旅顺公交\t辽宁阜新车牌\tbaidu\tk715\tk716...
3,6931EFC26D229CCFCEA125D3F3C21E57,4,2,3,最受欢迎狗狗排行榜\t舶怎么读\t场景描 写范例\t三维绘图软件\t枣和酸奶能一起吃吗\t好...
4,E780470C3BB0D340334BD08CDCC3C71A,2,2,4,干槽症能自愈吗\t太太万岁叶舒心去没去美国\t干槽症\t右眼皮下面一直跳是怎么回事\t麦当劳...
5,5BB424C7298B2CB67C368DDB73D4D336,4,2,0,钢琴曲欣赏100首\t一个月的宝宝眼睫毛那么是黄色\t宝宝右眼有眼屎\t小儿抽搐怎么办\t剖...
6,0D58894CD481D4B3039000661665C0DB,3,2,3,貔貅饲育指南\t韩姨\t贵阳学古琴\t洛基\t风速狗\t综英美]店主不好吃\t>重生之被拐儿...
7,0BA56489237BC91353300BE44298C6D8,6,1,0,昂的笔顺\t三年级上册数学口算\t牛的英语怎么读\t抓木机\t60米差不多跑多长时间\t似乎...
8,3D39C1B9312105E030B039CCDCD79611,1,1,5,李明霖在哪直播\t英文翻译\t薄荷籽\t为什么搜狐视频不能缓存\t有一首歌女生唱的歌词是冬天...
9,93174489E2E708BF6F797DC69F1531A8,1,1,5,bft20\t龙王传说\t长治到成都可以怎么坐火车\t龙王传说txt\t2 4 8 16老司...


## 查看用户数据,看数据均衡

In [108]:
df_train.info() # 数据没有缺失值
df_train[['Age','Gender','Education']].agg(['mean','sum','median','count']) 
for i in ['Age','Gender','Education']:
    df_train[i].value_counts()

# 数据在Age=1的比较多, Gender=1比较多,education在5比较多,可以看出样本总体不均衡,所以训练模型的时候需要切分

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
ID            100000 non-null object
Age           100000 non-null int64
Gender        100000 non-null int64
Education     100000 non-null int64
Query_List    100000 non-null object
dtypes: int64(3), object(2)
memory usage: 3.8+ MB


,Age,Gender,Education
mean,2.08231,1.38714,3.90333
sum,208231.00000,138714.00000,390333.00000
median,2.00000,1.00000,4.00000
count,100000.00000,100000.00000,100000.00000


1    38996
2    26744
3    18529
4    10654
5     2922
0     1666
6      489
Name: Age, dtype: int64

1    56976
2    40869
0     2155
Name: Gender, dtype: int64

5    37107
4    28148
3    18858
0     9280
6     5693
2      560
1      354
Name: Education, dtype: int64

In [109]:
np.mean(df_train['Query_List'].agg(lambda x: len(x)))

1232.69503

## 清洗数据

### 读取停用词

In [110]:
stopwords = pd.read_csv('./data/stop.txt',sep='\t',index_col=False,quoting=3,encoding='utf-8',names=['stopwords'])
stopwords = stopwords['stopwords'].values

### 清洗词函数

In [111]:
# 1 思路: 用正则匹配有http或者www, 看看都是什么类型的,然后用离散型数值表示
pattern_http = '((http|ftp|https):\/\/[\w\-_]+[\w\-_]+[\w\-\.,@?^=%&:/~\+#]*[\w\-\@?^=%&/~\+#]?)'
def http_cov(text):
    text = text.replace('ｗｗｗ．','www.').replace('．ｃｏｍ','.com').replace('\t','')
    text = re.sub(pattern_http,"WEB",text)
    return text
# 2 思路: 用正则匹配有邮箱地址, 然后用离散型数值表示
def email_cov(text):
    text = re.sub('([A-Za-z0-9\u4e00-\u9fa5]+@[a-zA-Z0-9_-]+(\.[a-zA-Z0-9_-]+))','EML',text)
    return text
#to do 可以把各个专业领域的专业用词导入,做分词,使用jieba.load_userdict(file_name), jieba.cut(ori, HMM=False),
# 词典格式和dict.txt一样，一个词占一行；每一行分三部分，一部分为词语，另一部分为词频，最后为词性（可省略），用空格隔开
# 宜信普惠 7
# 宜信 10
# 极速模式 20
# 汽车方面可以加入结巴,品牌对人群区分有影响
# 分词与词性过滤 (只保留n,v,j)

### 分词+词性,停用词筛选

In [112]:
import jieba
import jieba.analyse
import jieba.posseg
 
def dosegment_all(sentence):
    '''
    带词性标注，对句子进行分词，不排除停词等
    :param sentence:输入字符
    :return:
    '''
    allowPOS = ['n','v','j','eng']
    sentence_seged = jieba.posseg.cut(sentence.strip())
    outstr = ''
    for x in sentence_seged:
        if x.flag in allowPOS and len(x.word)>=2: 
            if x.word not in stopwords:
                outstr += '%s,'%(x.word)
#             print('yes')
#             outstr+="{}/{},".format(x.word,x.flag)
        else:
            pass
    return outstr
    #上面的for循环可以用python递推式构造生成器完成
    # outstr = ",".join([("%s/%s" %(x.word,x.flag)) for x in sentence_seged])

In [113]:
dosegment_all('苹果官网iPhone降价！WEB再次惊觉了神网友们的才华 一群同学 WLT, 记者说"sub",男子直接降掉0点了')

'苹果,官网,iPhone,降价,WEB,网友,同学,WLT,记者,降掉,'

In [114]:
df = df_train[df_train['Age'] != 0][df_train['Gender']!=0][df_train['Education']!=0]

In [115]:
df.info()
for i in ['Age','Gender','Education']:
    df[i].value_counts()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 88492 entries, 0 to 99999
Data columns (total 5 columns):
ID            88492 non-null object
Age           88492 non-null int64
Gender        88492 non-null int64
Education     88492 non-null int64
Query_List    88492 non-null object
dtypes: int64(3), object(2)
memory usage: 4.1+ MB


1    35965
2    24785
3    16561
4     8967
5     2039
6      175
Name: Age, dtype: int64

1    51540
2    36952
Name: Gender, dtype: int64

5    36852
4    27956
3    18708
6     4066
2      556
1      354
Name: Education, dtype: int64

In [125]:
df = df[:10000]
df
df_train.to_csv('./data/10rawQueryList.csv',index=False)

,ID,Age,Gender,Education,Query_List
0,22DD920316420BE2DF8D6EE651BA174B,1,1,4,柔和双沟女生中财网首页 财经WEB 司少的天价宝贝什么是遥控魔棒徽信表情动态搞笑图片教师节征...
1,43CC3AF5A8D6430A3B572337A889AFE4,2,1,3,"广州厨宝烤箱世情薄,人情恶,雨送黄昏花易落,晓风干,泪痕厦门酒店用品批发市场我只是不想让你支..."
3,6931EFC26D229CCFCEA125D3F3C21E57,4,2,3,最受欢迎狗狗排行榜舶怎么读场景描 写范例三维绘图软件枣和酸奶能一起吃吗好不壮观租衣服的网站动...
4,E780470C3BB0D340334BD08CDCC3C71A,2,2,4,干槽症能自愈吗太太万岁叶舒心去没去美国干槽症右眼皮下面一直跳是怎么回事麦当劳麦旋风的勺子吉林...
6,0D58894CD481D4B3039000661665C0DB,3,2,3,貔貅饲育指南韩姨贵阳学古琴洛基风速狗综英美]店主不好吃>重生之被拐儿童寻家记锦里oppor7...
...,...,...,...,...,...
11309,D7ABBB63488E74212C2768340831F6F2,3,2,4,护肤101个小窍门人绒毛膜促性腺激素高徐香猕猴桃图片励志句子唯美简短好用不贵的手机流产后第五...
11310,ED75AD35E1994A0376A9A67BECADC64E,4,1,3,2016年9月6日21点16分出生男孩武动乾坤漫画腾讯会员是不是qq会员百度妖精的尾巴漫画4...
11311,F0182F1C14DA747372688F1488587669,1,1,5,去黑头收缩毛孔小妙招甩葱歌原版馆陶黑陶韦正汪远塌鼻子g180次列车粉丝要泡多久台球入门手势乐...
11312,EA8FB33BA69F3FEA624E798DD0437034,2,2,5,漳州城厢区大麦若叶的功效与作用给旅游相册取个名字蓝胖子横屏麻辣变形计演员表闽西八大干atm机...


In [126]:
df['Query_List'] = df['Query_List'].apply(http_cov).apply(email_cov)

In [127]:
df

,ID,Age,Gender,Education,Query_List
0,22DD920316420BE2DF8D6EE651BA174B,1,1,4,柔和双沟女生中财网首页 财经WEB 司少的天价宝贝什么是遥控魔棒徽信表情动态搞笑图片教师节征...
1,43CC3AF5A8D6430A3B572337A889AFE4,2,1,3,"广州厨宝烤箱世情薄,人情恶,雨送黄昏花易落,晓风干,泪痕厦门酒店用品批发市场我只是不想让你支..."
3,6931EFC26D229CCFCEA125D3F3C21E57,4,2,3,最受欢迎狗狗排行榜舶怎么读场景描 写范例三维绘图软件枣和酸奶能一起吃吗好不壮观租衣服的网站动...
4,E780470C3BB0D340334BD08CDCC3C71A,2,2,4,干槽症能自愈吗太太万岁叶舒心去没去美国干槽症右眼皮下面一直跳是怎么回事麦当劳麦旋风的勺子吉林...
6,0D58894CD481D4B3039000661665C0DB,3,2,3,貔貅饲育指南韩姨贵阳学古琴洛基风速狗综英美]店主不好吃>重生之被拐儿童寻家记锦里oppor7...
...,...,...,...,...,...
11309,D7ABBB63488E74212C2768340831F6F2,3,2,4,护肤101个小窍门人绒毛膜促性腺激素高徐香猕猴桃图片励志句子唯美简短好用不贵的手机流产后第五...
11310,ED75AD35E1994A0376A9A67BECADC64E,4,1,3,2016年9月6日21点16分出生男孩武动乾坤漫画腾讯会员是不是qq会员百度妖精的尾巴漫画4...
11311,F0182F1C14DA747372688F1488587669,1,1,5,去黑头收缩毛孔小妙招甩葱歌原版馆陶黑陶韦正汪远塌鼻子g180次列车粉丝要泡多久台球入门手势乐...
11312,EA8FB33BA69F3FEA624E798DD0437034,2,2,5,漳州城厢区大麦若叶的功效与作用给旅游相册取个名字蓝胖子横屏麻辣变形计演员表闽西八大干atm机...


In [128]:
df['Query_List'] = df['Query_List'].apply(dosegment_all)

In [130]:
df.to_csv('./data/Cut1WQueryList.csv',index=False)
pd.read_csv('./data/Cut1WQueryList.csv')

,ID,Age,Gender,Education,Query_List
0,22DD920316420BE2DF8D6EE651BA174B,1,1,4,"双沟,女生,财经,WEB,天价,魔棒,表情,动态,搞笑,图片,教师节,征文,营业厅,校草,未..."
1,43CC3AF5A8D6430A3B572337A889AFE4,2,1,3,"厨宝,烤箱,世情,人情,花易落,风干,泪痕,酒店用品,批发市场,不想,支付,无情,处女座,代..."
2,6931EFC26D229CCFCEA125D3F3C21E57,4,2,3,"受欢迎,狗狗,排行榜,场景,范例,绘图,软件,酸奶,衣服,网站,动漫,绘图,软件,妒羡,瀑布..."
3,E780470C3BB0D340334BD08CDCC3C71A,2,2,4,"干槽症,太太,没去,干槽症,眼皮,旋风,勺子,制药,股份,有限公司,脑络,胶囊,蛐蛐,蜂蜜,..."
4,0D58894CD481D4B3039000661665C0DB,3,2,3,"貔貅,饲育,指南,风速,店主,好吃,重生,儿童,oppor7,领养,神探,豪门,贵妇,守则,..."
...,...,...,...,...,...
9995,D7ABBB63488E74212C2768340831F6F2,3,2,4,"护肤,绒毛,猕猴桃,图片,励志,句子,简短,好用,手机,出血,增多,血块,绒毛,手机,官网,..."
9996,ED75AD35E1994A0376A9A67BECADC64E,4,1,3,"出生,男孩,武动,漫画,会员,qq,会员,百度,妖精,尾巴,漫画,手游,出生,baidu,漫..."
9997,F0182F1C14DA747372688F1488587669,1,1,5,"黑头,收缩,毛孔,原版,馆陶,塌鼻子,g180,次列车,粉丝,台球,手势,乐视,手机,刷机,..."
9998,EA8FB33BA69F3FEA624E798DD0437034,2,2,5,"大麦,功效,作用,相册,名字,胖子,横屏,变形,演员表,八大,atm,天猫,超市,快递,部队..."


In [93]:
import jieba
# # import jieba.analyse
# df_train['Query_List'] = df_train['Query_List'].apply(dosegment_all)

In [ ]:
df_train['Query_List']

In [ ]:
df_train.to_csv('./data/10posgQueryList.csv',index=False)

In [50]:
# import jieba
# import jieba.analyse
# sentences = jieba.lcut(df_train['Query_List'][0])
# # jieba.analyse.extract_tags(sentences,topK=20)
# sentences = ' '.join(sentences)
# sentences.strip()
# jieba.analyse.extract_tags(sentences,topK=20)

In [106]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score,precision_score
import numpy as np
from sklearn.naive_bayes import MultinomialNB,BernoulliNB
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.model_selection import train_test_split

In [93]:
df_train['Query_List'] = df_train['Query_List'].apply(lambda x : ' '.join(x))

In [102]:
X = df_train['Query_List'].values.tolist()
Y = df_train['Age'].values.tolist()
x_train,x_test,y_train,y_test = train_test_split(X,Y,random_state=22)
vec = TfidfVectorizer(analyzer='word',max_features=10000)
vec.fit(X)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=10000,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [ ]:
def stratifiedkfold_cv(x,y,clf_class,shuffle=True,n_folds=5,**kwargs):
    stratifiedk_fold=StratifiedKFold(n_splits=n_folds,shuffle=shuffle)
    y_pred = y[:]
    for train_index,test_index in stratifiedk_fold.split(x,y):
        X_train,X_test = x[train_index],x[test_index]
        y_train= y[train_index]
        clf = clf_class(**kwargs)
        clf.fit(X_train,y_train)
        y_pred[test_index] = clf.predict(X_test)
    return y_pred
NB = MultinomialNB
BR = BernoulliNB
for i in [NB,BR]:
    print(precision_score(Y,stratifiedkfold_cv(vec.transform(X),np.array(Y),i),average='macro'))

0.26819080442394533


In [ ]:
from gensim.models import word2vec